# Twitter Train
### Twitter train assignment from https://inclass.kaggle.com/c/twitter-sentiment-analysis2

### Download Data

In [1]:
%%bash 
rm -r Data
mkdir Data
cd Data
kg download -c "twitter-sentiment-analysis2"

downloading https://www.kaggle.com/c/twitter-sentiment-analysis2/download/train.csv

downloading https://www.kaggle.com/c/twitter-sentiment-analysis2/download/test.csv



rm: Data: No such file or directory
train.csv 100% |#####################################| Time: 0:00:04   1.7 MiB/s
test.csv 100% |######################################| Time: 0:00:12   2.1 MiB/s


## Pre -Process data:
1. Take csv data parase and create a word to vec model.
2. Take the data and convert the Sentiment Text with word2vec vector arrays.
3. Pad the data.

In [2]:
import pandas as pd
# from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
raw_data = pd.read_csv('./Data/train.csv').drop("ItemID", axis=1)
test_data = pd.read_csv('./Data/test.csv')
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.embeddings import Embedding
from keras.layers import SpatialDropout1D
from keras.layers.recurrent import LSTM
EPOCHS = 1


Using TensorFlow backend.


In [3]:
raw_data.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [4]:
tokenized = []
for x in raw_data['SentimentText'].values.tolist():
    tokenized.append(
        text_to_word_sequence(
            x,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            split=" ",
            lower=True
        ))

for x in test_data['SentimentText'].values.tolist():
    tokenized.append(
        text_to_word_sequence(
            x,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            split=" ",
            lower=True
        ))
tweet_model = Word2Vec(tokenized, min_count=1, size=100)
tweet_model.save("tweetmodel")

In [5]:
tweet_model.wv.most_similar("hello")

[('hi', 0.8925399780273438),
 ('hey', 0.6679447889328003),
 ('alanacoops', 0.6002880930900574),
 ('hiya', 0.5733997821807861),
 ('mg8k3d', 0.5646853446960449),
 ('goodbye', 0.5546966791152954),
 ('jeremyroloff', 0.5459803938865662),
 ('trappeded', 0.545304536819458),
 ('billyraycyrus', 0.5431361794471741),
 ('mr', 0.5423548221588135)]

### Convert data to set of vector arrays

In [6]:
def processtext(x):
    k = text_to_word_sequence(
        x,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        split=" ",
        lower=True
    )
    k_vec = []
    global count
#     count = len(k) if len(k) > count else count
    for word in k:
        try:
            vec = (tweet_model.wv[word])
        except KeyError:
            print(k)
        else:
            k_vec.append(vec)
    return k_vec
print("Processing data...")
x = map(lambda seq: processtext(seq), raw_data['SentimentText'].values)

Processing data...


In [7]:
x[0][0].shape

(100,)

### Pad data to create a tensorflow compatible numpy matrix

In [8]:
x = pad_sequences(x)
y = raw_data['Sentiment'].values
# x_test = pad_sequences(x_test)

In [10]:
x.shape

(99989, 151, 100)

In [11]:
y.shape

(99989,)

### Create a model

In [ ]:
print("Building Model...")
max_features = 20000

model = Sequential()
model.add(Dense(200, input_shape=x[0].shape))
model.add(SpatialDropout1D(rate=0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=["accuracy"])

Building Model...


### Train Model

In [ ]:
print("Training Model")
model.fit(x, y=y, epochs=EPOCHS, verbose=1,
          validation_split=0.2, shuffle=True)
print("Model Trained")

Training Model
Train on 79991 samples, validate on 19998 samples
Epoch 1/1
19552/79991 [======>.......................] - ETA: 1420s - loss: 0.5514 - acc: 0.7131

In [ ]:
# model.predict(x=x_test, y=y_test)

In [ ]:
# model.save("twitter-sentiment-analysis2.h5")